In [3]:
import pandas as pd
import torch
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from torch.utils.data import Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

drive.mount('/content/drive')

Device: cuda
Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1JuvXZ8vO66_Vje9xLUzqnhe6nupbRU63/CS584/even_split_cleaned_unicode.csv')

df.head(10)

,Unnamed: 0,tickers,companyName,accessionNumber,document,filingDate,url,content,price_today,price_3mo,price_change,direction
0,1,MMM,3M CO,0000066740-25-000063,mmm-20250630.htm,2025-07-18,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,153.229996,152.639999,-0.003850,0
1,4,MMM,3M CO,0000066740-24-000080,mmm-20240630.htm,2024-07-26,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,127.160004,124.750000,-0.018953,0
2,7,MMM,3M CO,0000066740-23-000058,mmm-20230630.htm,2023-07-25,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,91.831100,74.740807,-0.186106,0
3,9,MMM,3M CO,0000066740-22-000076,mmm-20220930.htm,2022-10-25,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,99.080269,94.423080,-0.047004,0
4,10,MMM,3M CO,0000066740-22-000065,mmm-20220630.htm,2022-07-27,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,116.103676,102.675583,-0.115656,0
5,11,MMM,3M CO,0000066740-22-000036,mmm-20220331.htm,2022-04-26,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,120.585281,117.683945,-0.024060,0
6,12,MMM,3M CO,0000066740-21-000013,mmm-20210930.htm,2021-10-26,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,152.307693,141.555191,-0.070597,0
7,13,MMM,3M CO,0001558370-21-009338,mmm-20210630x10q.htm,2021-07-27,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,167.617050,149.030106,-0.110889,0
8,17,MMM,3M CO,0001558370-20-004478,mmm-20200428x10q.htm,2020-04-28,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,131.780930,129.874588,-0.014466,0
9,19,MMM,3M CO,0001558370-19-006397,mmm-20190630x10q.htm,2019-07-26,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management's Discussion and Analysis o...,145.468231,138.871231,-0.045350,0


In [5]:
df.shape

(7746, 12)

In [7]:
df2 = pd.read_csv('/content/drive/.shortcut-targets-by-id/1JuvXZ8vO66_Vje9xLUzqnhe6nupbRU63/CS584/finbert_output.csv')


In [8]:
df2.head(10)

,Unnamed: 0,tickers,companyName,accessionNumber,document,filingDate,url,content,price_today,price_3mo,price_change,direction,positive,negative,neutral
0,1,MMM,3M CO,0000066740-25-000063,mmm-20250630.htm,2025-07-18,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,153.229996,152.639999,-0.003850,0,0.132291,0.303619,0.564090
1,2,MMM,3M CO,0000066740-25-000039,mmm-20250331.htm,2025-04-22,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,136.330002,151.199997,0.109074,1,0.095642,0.234331,0.670027
2,3,MMM,3M CO,0000066740-24-000101,mmm-20240930.htm,2024-10-22,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,131.729996,149.119995,0.132012,1,0.059686,0.325514,0.614801
3,4,MMM,3M CO,0000066740-24-000080,mmm-20240630.htm,2024-07-26,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,127.160004,124.750000,-0.018953,0,0.082315,0.346704,0.570981
4,5,MMM,3M CO,0000066740-24-000053,mmm-20240331.htm,2024-04-30,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,96.510002,126.750000,0.313335,1,0.117315,0.241281,0.641404
5,6,MMM,3M CO,0000066740-23-000092,mmm-20230930.htm,2023-10-24,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,75.351173,77.959869,0.034621,1,0.057124,0.364949,0.577927
6,7,MMM,3M CO,0000066740-23-000058,mmm-20230630.htm,2023-07-25,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,91.831100,74.740807,-0.186106,0,0.047720,0.419279,0.533001
7,8,MMM,3M CO,0000066740-23-000028,mmm-20230331.htm,2023-04-25,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,87.265884,91.831100,0.052314,1,0.083089,0.294037,0.622874
8,9,MMM,3M CO,0000066740-22-000076,mmm-20220930.htm,2022-10-25,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,99.080269,94.423080,-0.047004,0,0.067711,0.358310,0.573980
9,10,MMM,3M CO,0000066740-22-000065,mmm-20220630.htm,2022-07-27,https://www.sec.gov/Archives/edgar/data/000006...,Item 2. Management’s Discussion and Analysis o...,116.103676,102.675583,-0.115656,0,0.085860,0.387758,0.526382


In [28]:
accession_num_to_check = df.iloc[840]['accessionNumber']
filing_date_to_check = df.iloc[840]['filingDate']
display(df[(df['accessionNumber'] == accession_num_to_check) & (df['filingDate'] == filing_date_to_check)])

display(df2[(df2['accessionNumber'] == accession_num_to_check) & (df2['filingDate'] == filing_date_to_check)])


,Unnamed: 0,tickers,companyName,accessionNumber,document,filingDate,url,content,price_today,price_3mo,price_change,direction,positive,negative,neutral
840,2958,CME,CME GROUP INC.,0001156375-22-000158,cme-20220331.htm,2022-05-04,https://www.sec.gov/Archives/edgar/data/000115...,ITEM 2.MANAGEMENT'S DISCUSSION AND ANALYSIS OF...,220.660004,200.029999,-0.093492,0,0.110539,0.3185,0.570961


,Unnamed: 0,tickers,companyName,accessionNumber,document,filingDate,url,content,price_today,price_3mo,price_change,direction,positive,negative,neutral
2249,2958,CME,CME GROUP INC.,0001156375-22-000158,cme-20220331.htm,2022-05-04,https://www.sec.gov/Archives/edgar/data/000115...,ITEM 2.MANAGEMENT’S DISCUSSION AND ANALYSIS OF...,220.660004,200.029999,-0.093492,0,0.110539,0.3185,0.570961


In [30]:
output_path = '/content/drive/MyDrive/CS584/balanced_data_sentiment.csv'
df.to_csv(output_path, index=False)

In [31]:
df.shape

(7746, 15)

In [38]:
print((df['direction'] == 0).sum())
print((df['direction'] == 1).sum())

3873
3873
